In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sympy
from re import findall

This notebook develops an ODE solver that follows the evolution of the nine species of $\rm{H}$, $\rm{H^+}$, $\rm{H^-}$, $\rm{He}$, $\rm{He^+}$, $\rm{He^{++}}$, $\rm{H_2}$, $\rm{H_2^+}$ and $\rm{e^-}$. The nine species network include the following reactions ([arXiv:1610.09591](https://arxiv.org/abs/1610.09591)):
- ${\rm H + e^{-}} \rightarrow \rm{H^{+} + e^{-} + e^{-}}$
- ${\rm H^{+} + e^{-}} \rightarrow \rm{H + \gamma} $
- ${\rm He + e^{-}} \rightarrow {\rm He^{+} + e^{-} + e^{-}}$
- ${\rm He^{+} + e^{-}} \rightarrow {\rm He + \gamma}$
- ${\rm He^{+} + e^{-}} \rightarrow {\rm He^{++} + e^{-} + e^{-}}$
- ${\rm He^{++} + e^{-}} \rightarrow {\rm He^{+} + \gamma}$
- ${\rm H + H} \rightarrow {\rm H^{+} + e^{-} + H}$
- ${\rm H + He} \rightarrow {\rm H^{+} + e^{-} + He}$
- ${\rm H + \gamma} \rightarrow {\rm H^{+} + e^{-}}$
- ${\rm He + \gamma} \rightarrow {\rm He^{+} + e^{-}}$
- ${\rm He^{+} + \gamma} \rightarrow {\rm He^{++} + e^{-}}$
- ${\rm H + e^{-}} \rightarrow {\rm H^{-} + \gamma}$
- ${\rm H^{-} + H} \rightarrow {\rm H_{2} + e^{-}}$
- ${\rm H + H^{+}} \rightarrow {\rm H_{2}^{+} + \gamma}$
- ${\rm H_{2}^{+} + H} \rightarrow {\rm H_{2} + H^{+}}$
- ${\rm H_{2} + H^{+}} \rightarrow {\rm H_{2}^{+} + H}$
- ${\rm H_{2} + e^{-}} \rightarrow {\rm H + H + e^{-}}$
- ${\rm H_{2} + H} \rightarrow {\rm H + H + H}$
- ${\rm H^{-} + e^{-}} \rightarrow {\rm H + e^{-} + e^{-}}$
- ${\rm H^{-} + H} \rightarrow {\rm H + e^{-} + H}$
- ${\rm H^{-} + H^{+}} \rightarrow {\rm H + H}$
- ${\rm H^{-} + H^{+}} \rightarrow {\rm H_{2}^{+} + e^{-}}$
- ${\rm H_{2}^{+} + e^{-}} \rightarrow {\rm H + H}$
- ${\rm H_{2}^{+} + H^{-}} \rightarrow {\rm H_{2} + H}$
- ${\rm H + H + H} \rightarrow {\rm H_{2} + H}$
- ${\rm H + H + H_{2}} \rightarrow {\rm H_{2} + H_{2}}$
- ${\rm H^{-} + \gamma} \rightarrow {\rm H + e^{-}}$
- ${\rm H_{2}^{+} + \gamma} \rightarrow {\rm H + H^{+}}$
- ${\rm H_{2} + \gamma} \rightarrow {\rm H_{2}^{+} + e^{-}}$
- ${\rm H_{2}^{+} + \gamma} \rightarrow {\rm H^{+} + H^{+} + e^{-}}$
- ${\rm H_{2} + \gamma} \rightarrow {\rm H + H}$
- ${\rm H + H + grain} \rightarrow {\rm H_{2} + grain}$

In [2]:
all_species = sympy.sympify(
    'H, Hp, Hm, He, Hep, Hepp, H2, H2p, de, gma, grn'
)

H, Hp, Hm, He, Hep, Hepp, H2, H2p, de, gma, grn = all_species

k = sympy.sympify(
    'k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12, ' \
    'k13, k14, k15, k16, k17, k18, k19, k20, k21, k22, k23, k24, ' \
    'k25, k26, k27, k28, k29, k30, k31, k32'
)

Generate equations from the paper's $\LaTeX$ source:

In [3]:
source = '''- ${\rm H + e^{-}} \rightarrow {\rm H^{+} + e^{-} + e^{-}}$
- ${\rm H^{+} + e^{-}} \rightarrow {\rm H + \gamma}$
- ${\rm He + e^{-}} \rightarrow {\rm He^{+} + e^{-} + e^{-}}$
- ${\rm He^{+} + e^{-}} \rightarrow {\rm He + \gamma}$
- ${\rm He^{+} + e^{-}} \rightarrow {\rm He^{++} + e^{-} + e^{-}}$
- ${\rm He^{++} + e^{-}} \rightarrow {\rm He^{+} + \gamma}$
- ${\rm H + H} \rightarrow {\rm H^{+} + e^{-} + H}$
- ${\rm H + He} \rightarrow {\rm H^{+} + e^{-} + He}$
- ${\rm H + \gamma} \rightarrow {\rm H^{+} + e^{-}}$
- ${\rm He + \gamma} \rightarrow {\rm He^{+} + e^{-}}$
- ${\rm He^{+} + \gamma} \rightarrow {\rm He^{++} + e^{-}}$
- ${\rm H + e^{-}} \rightarrow {\rm H^{-} + \gamma}$
- ${\rm H^{-} + H} \rightarrow {\rm H_{2} + e^{-}}$
- ${\rm H + H^{+}} \rightarrow {\rm H_{2}^{+} + \gamma}$
- ${\rm H_{2}^{+} + H} \rightarrow {\rm H_{2} + H^{+}}$
- ${\rm H_{2} + H^{+}} \rightarrow {\rm H_{2}^{+} + H}$
- ${\rm H_{2} + e^{-}} \rightarrow {\rm H + H + e^{-}}$
- ${\rm H_{2} + H} \rightarrow {\rm H + H + H}$
- ${\rm H^{-} + e^{-}} \rightarrow {\rm H + e^{-} + e^{-}}$
- ${\rm H^{-} + H} \rightarrow {\rm H + e^{-} + H}$
- ${\rm H^{-} + H^{+}} \rightarrow {\rm H + H}$
- ${\rm H^{-} + H^{+}} \rightarrow {\rm H_{2}^{+} + e^{-}}$
- ${\rm H_{2}^{+} + e^{-}} \rightarrow {\rm H + H}$
- ${\rm H_{2}^{+} + H^{-}} \rightarrow {\rm H_{2} + H}$
- ${\rm H + H + H} \rightarrow {\rm H_{2} + H}$
- ${\rm H + H + H_{2}} \rightarrow {\rm H_{2} + H_{2}}$
- ${\rm H^{-} + \gamma} \rightarrow {\rm H + e^{-}}$
- ${\rm H_{2}^{+} + \gamma} \rightarrow {\rm H + H^{+}}$
- ${\rm H_{2} + \gamma} \rightarrow {\rm H_{2}^{+} + e^{-}}$
- ${\rm H_{2}^{+} + \gamma} \rightarrow {\rm H^{+} + H^{+} + e^{-}}$
- ${\rm H_{2} + \gamma} \rightarrow {\rm H + H}$
- ${\rm H + H + grain} \rightarrow {\rm H_{2} + grain}$'''

formulas = findall('\${\\rm (.*)} \\rightarrow {\\rm (.*)}\$', source)

replace_rules = [('e^{-}', 'de'), ('^{+}', 'p'), ('^{++}', 'pp'), ('^{-}', 'm'), 
                 ('_{2}', '2'), ('\\gamma', 'gma'), ('grain', 'grn')]

formulas_clean = []
for i in range(len(formulas)):
    lhs, rhs = formulas[i]
    for latex, plain in replace_rules:
        lhs = lhs.replace(latex, plain)
        rhs = rhs.replace(latex, plain)
    formulas_clean.append((lhs, rhs))

all_reactions = [
    eval('({lhs}), ({rhs}), k[{i}]'.format(lhs=lhs, rhs=rhs, i=i))
    for i, (lhs, rhs) in enumerate(formulas_clean)
]

In [4]:
# all_reactions = all_reactions[:2]

In [5]:
def find_formation(species):
    return [rxn for rxn in all_reactions if species in rxn[1].atoms()]

def find_destruction(species):
    return [rxn for rxn in all_reactions if species in rxn[0].atoms()]

In [6]:
def get_rhs(species):
    dSdt = 0
    for lhs, rhs, coef in find_formation(species):
        term = coef
        for atom in list(lhs.atoms()):
            term *= atom
        dSdt += term
    for lhs, rhs, coef in find_destruction(species):
        term = -coef
        for atom in list(lhs.atoms()):
            term *= atom
        dSdt += term
    return dSdt

In [7]:
rhs = {species: get_rhs(species) for species in all_species}

In [8]:
rhs

{H: -2*H*H2*k26 - H*H2p*k15 - H*He*k8 - H*Hm*k13 - H*Hp*k14 - H*de*k1 - H*de*k12 - H*gma*k9 - 2*H*grn*k32 + H2*Hp*k16 + H2*de*k17 + H2*gma*k31 + H2p*Hm*k24 + H2p*de*k23 + H2p*gma*k28 + Hm*Hp*k21 + Hm*de*k19 + Hm*gma*k27 + Hp*de*k2,
 H2: -H*H2*k18 + H*H2p*k15 + H*Hm*k13 + 2*H*grn*k32 + 3*H*k25 - H2*Hp*k16 - H2*de*k17 - H2*gma*k29 - H2*gma*k31 + H2p*Hm*k24,
 H2p: -H*H2p*k15 + H*Hp*k14 + H2*Hp*k16 + H2*gma*k29 - H2p*Hm*k24 - H2p*de*k23 - H2p*gma*k28 - H2p*gma*k30 + Hm*Hp*k22,
 He: -He*de*k3 - He*gma*k10 + Hep*de*k4,
 Hep: He*de*k3 + He*gma*k10 - Hep*de*k4 - Hep*de*k5 - Hep*gma*k11 + Hepp*de*k6,
 Hepp: Hep*de*k5 + Hep*gma*k11 - Hepp*de*k6,
 Hm: -H*Hm*k13 - H*Hm*k20 + H*de*k12 - H2p*Hm*k24 - Hm*Hp*k21 - Hm*Hp*k22 - Hm*de*k19 - Hm*gma*k27,
 Hp: H*H2p*k15 + H*He*k8 - H*Hp*k14 + H*de*k1 + H*gma*k9 + 2*H*k7 - H2*Hp*k16 + H2p*gma*k28 + H2p*gma*k30 - Hm*Hp*k21 - Hm*Hp*k22 - Hp*de*k2,
 de: H*He*k8 + H*Hm*k13 + H*Hm*k20 - H*de*k12 + H*gma*k9 + 2*H*k7 + H2*gma*k29 - H2p*de*k23 + H2p*gma*k30 + He*gma